In [7]:
import os
import pandas as pd
import numpy as np
import re

In [1]:
# Manual curation of which clustering threshold to use!

human_host = {"HA": 0.98, "NA": 0.98, "M1": 0.995, "M2": 0.98, "PA": 0.99, 
              "PB1":0.99, "PB2": 0.99, "NP":0.99, "NS1":0.98, "NS2":0.99}

avswine_host = {"HA": 0.95, "NA": 0.95, "M1": 0.99, "M2": 0.96, "PA": 0.97, 
                "PB1":0.96, "PB2": 0.97, "NP":0.96, "NS1":0.95, "NS2":0.97}

avswine_tree_host = {"HA": 0.85, "NA": 0.85, "M1": 0.96, "M2": 0.91, "PA": 0.96, 
                     "PB1": 0.94, "PB2": 0.96, "NP":0.94, "NS1":0.88, "NS2":0.92}

In [2]:
filter_cutoff = {"HA": 90, "NA": 90, "M1": 90, "M2": 90, "PB1":99.5, "PB2":99, "NP":95, "NS1":90, "NS2":90, "PA":99}

In [112]:
prot = "M1"
h_clust, avs_clust = int(human_host[prot]*100), int(avswine_host[prot]*100)

In [113]:
h_known_epitopes = pd.read_csv("../sequences/epitope_conservancy/{}_AB_clust{}_conservancy.csv".format(prot, h_clust))
avs_known_epitopes = pd.read_csv("../sequences/epitope_conservancy/{}_AS_clust{}_conservancy.csv".format(prot, avs_clust))

In [114]:
h_known_epitopes_filter1 = h_known_epitopes.loc[h_known_epitopes["Fraction of 100% matching sequences (%)"] >= 50]

In [115]:
h_known_epitopes_filter2 = h_known_epitopes_filter1.loc[h_known_epitopes_filter1["Fraction of <50% matching sequences (%)"] <= 20]

In [116]:
avs_known_epitopes_filter1 = avs_known_epitopes.loc[avs_known_epitopes["Fraction of 100% matching sequences (%)"] >= 50]

In [117]:
avs_known_epitopes_filter2 = avs_known_epitopes_filter1.loc[avs_known_epitopes["Fraction of <50% matching sequences (%)"] <= 20]

In [120]:
overlap_known_epitopes = []

for i in h_known_epitopes_filter2['Epitope sequence']:
    for j in avs_known_epitopes_filter2['Epitope sequence']:
        if i == j:
            overlap_known_epitopes.append(j)

len(overlap_known_epitopes)

6

In [211]:
prot = "PA"
h_clust, avs_clust = int(human_host[prot]*100), int(avswine_host[prot]*100)

h_exp_epitopes_table = pd.read_csv("../sequences/nonoverlap_candidates/{}_AB_clust{}_candidate_epitopes.csv".format(prot, h_clust))
avs_exp_epitopes_table = pd.read_csv("../sequences/nonoverlap_candidates/{}_AS_clust{}_candidate_epitopes.csv".format(prot, avs_clust))

In [212]:
h_exp_epitopes_table_filter1 = h_exp_epitopes_table.loc[h_exp_epitopes_table['score (%)'] >= filter_cutoff[prot]]
avs_exp_epitopes_table_filter1 = avs_exp_epitopes_table.loc[avs_exp_epitopes_table['score (%)'] >= filter_cutoff[prot]]

In [213]:
h_exp_epitopes = h_exp_epitopes_table_filter1['candidate sequence']
avs_exp_epitopes = avs_exp_epitopes_table_filter1['candidate sequence']

In [214]:
overlap_exp_epitopes = []

for hexp in h_exp_epitopes:
    for avsexp in avs_exp_epitopes:
        if hexp == avsexp:
            overlap_exp_epitopes.append(avsexp)

len(overlap_exp_epitopes)

89

In [208]:
# make separate files according to their lengths for netMHCpan prediction
known_mers = sorted(set(h_known_epitopes_filter2['Epitope length']))
exp_mers = sorted(set(h_exp_epitopes_table_filter1['length']))

outpath = "../netmhc_binding_inputs/{}".format(prot)
if not os.path.exists(outpath):
    os.mkdir(outpath)

for mer in known_mers:
    known_fasta = "{}/{}_conserved_{}_mers.fa".format(outpath, prot, mer)
    with open(known_fasta, 'w') as output:
        for i, seq in enumerate(h_known_epitopes_filter2['Epitope sequence']):
            if len(seq) == mer:
                if seq in overlap_known_epitopes:
                    output.write(">cons_{}_o \n".format(i))
                    output.write(seq + '\n')
                else:
                    output.write(">cons_{} \n".format(i))
                    output.write(seq + '\n')

for mer in exp_mers:
    exp_fasta = "{}/{}_exp_{}_mers.fa".format(outpath, prot, mer)
    with open(exp_fasta, 'w') as output:
        for i, seq in enumerate(h_exp_epitopes_table_filter1['candidate sequence']):
            if len(seq) == mer:
                if seq in overlap_exp_epitopes:
                    output.write(">cand_{}_o \n".format(i))
                    output.write(seq + '\n')
                else:
                    output.write(">cand_{} \n".format(i))
                    output.write(seq + '\n')